In [ ]:
import matplotlib.animation as animation
import numpy as np
from numpy.linalg import eigh, eig
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go

In [ ]:
class Map:
    def __init__(self, src):
        raw_data = np.loadtxt(src, delimiter='\t')
        self.raman_shift = raw_data.T[1]
        self.map = []
        for row in raw_data.T:
            self.map.append({
                'x': row[0],
                'y': row[1],
                'spectrum': row[1:]
            })
        self.cov_matrix = np.cov(np.array([x['spectrum'] for x in self.map]))
        
        eigenvalues, eigenvectors = eigh(self.cov_matrix)
        idx = np.flip(eigenvalues.argsort())
        self.weights = eigenvalues[idx]
        self.vectors = eigenvectors[:,idx]
        for i in range(len(self.vectors)):
            self.vectors[i]/=np.dot(self.vectors[i], self.vectors[i])**.5

    def plotCovMatrix(self):
        fig = px.imshow(self.cov_matrix, title = 'Macierz kowariancji')
        return fig
    
    def plotWeights(self):
        fig = px.bar(self.weights, title = 'Wagi')
        return fig
    
    
        
map_a = Map('GA25_10B_532nm_100%_1x3sec_x100_xc1200_A_trojkat1_mapa_10x10um_step_0.3um_data.txt')

map_a.plotCovMatrix().show()
map_a.plotWeigths().show()